In [666]:
import pandas as pd
import numpy as np

## Data Cleaning

### Brief Summary: 
Due to the pre-cleaned nature of the data, the dataset utilized for this analysis consists of no duplicate observations as well as no missing values. Thus, the cleaning process only consists of selecting relevant rows and columns to conduct time series forecasting using both univariate and multivariate datasets.

### Remove Irrelevant Columns

In [667]:
#Load the data
data=pd.read_csv(r"C:\Users\hajar\Time-Series-Crime-Forecasting-Minneapolis-3\data\Crime_Data.csv")

#Remove irrelevant columns
cols_to_be_deleted = [
    "Case_Number", "Case_NumberAlt", "Occurred_Date",
    "NIBRS_Code","Problem_Initial", "Problem_Final","Address",
    "wgsXAnon","wgsYAnon", "X","Y","OBJECTID","Offense","NIBRS_Group",
    "Latitude","Longitude", "Neighborhood", "Precinct"
]

data=data.drop(columns=cols_to_be_deleted)
data

,Type,Reported_Date,NIBRS_Crime_Against,Offense_Category,Ward,Crime_Count
0,Additional Crime Metrics,2019/01/05 02:50:00+00,Non NIBRS Data,Subset of NIBRS Assault Offenses,6.0,1
1,Additional Crime Metrics,2019/01/08 12:22:00+00,Non NIBRS Data,Subset of NIBRS Assault Offenses,4.0,1
2,Additional Crime Metrics,2019/01/10 14:10:00+00,Non NIBRS Data,Subset of NIBRS Assault Offenses,5.0,1
3,Additional Crime Metrics,2019/01/27 06:52:00+00,Non NIBRS Data,Subset of NIBRS Assault Offenses,4.0,1
4,Additional Crime Metrics,2019/02/03 03:24:00+00,Non NIBRS Data,Subset of NIBRS Assault Offenses,1.0,1
...,...,...,...,...,...,...
323390,Shots Fired Calls,2025/03/07 00:20:43+00,Non NIBRS Data,Shots Fired Calls,5.0,1
323391,Shots Fired Calls,2025/03/07 14:24:33+00,Non NIBRS Data,Shots Fired Calls,4.0,1
323392,Shots Fired Calls,2025/03/08 00:12:37+00,Non NIBRS Data,Shots Fired Calls,2.0,1
323393,Shots Fired Calls,2025/03/09 19:44:43+00,Non NIBRS Data,Shots Fired Calls,4.0,1


### Remove Irrelevant Rows

In [668]:
data = data[data.Type =="Crime Offenses (NIBRS)"]
data = data[data.NIBRS_Crime_Against != "Not a Crime "]
data=data.drop(columns=['Type']) #Now all the observations are Crime Offenses

data

,Reported_Date,NIBRS_Crime_Against,Offense_Category,Ward,Crime_Count
15,2019/01/01 07:53:00+00,Person,Assault Offenses,10.0,1
16,2019/01/04 10:07:00+00,Property,Destruction/Damage/Vandalism of Property,3.0,1
17,2019/01/01 03:23:00+00,Property,Destruction/Damage/Vandalism of Property,1.0,1
33,2019/01/02 04:04:00+00,Property,Larceny/Theft Offenses,1.0,1
34,2019/01/04 05:27:00+00,Property,Stolen Property Offenses,4.0,1
...,...,...,...,...,...
318591,2025/03/06 21:19:00+00,Property,Larceny/Theft Offenses,10.0,1
318592,2025/03/07 13:04:00+00,Property,Larceny/Theft Offenses,12.0,1
318593,2025/03/08 11:57:00+00,Property,Destruction/Damage/Vandalism of Property,8.0,1
318594,2025/03/10 11:44:00+00,Property,Larceny/Theft Offenses,8.0,1


### Remove Duplicates and Expand Data

In [669]:
print ("Rows     : " ,data.shape[0])
print ("Columns  : " ,data.shape[1])
print ("\nFeatures : \n\n" ,data.columns.tolist())
print ("\nMissing values : \n\n", data.isnull().any())
print ("\nUnique values :  \n\n",data.nunique())

Rows     :  266185
Columns  :  5

Features : 

 ['Reported_Date', 'NIBRS_Crime_Against', 'Offense_Category', 'Ward', 'Crime_Count']

Missing values : 

 Reported_Date          False
NIBRS_Crime_Against    False
Offense_Category       False
Ward                    True
Crime_Count            False
dtype: bool

Unique values :  

 Reported_Date          234248
NIBRS_Crime_Against         3
Offense_Category           23
Ward                       13
Crime_Count                15
dtype: int64


In [670]:
category_counts = data['Crime_Count'].value_counts()
category_counts

Crime_Count
1     259930
2       4857
3        854
4        313
5        122
6         40
7         32
8         13
9          8
10         5
12         4
11         3
14         2
13         1
21         1
Name: count, dtype: int64

### Convert Reported_Date to datetime type and make a Date Time Column

In [671]:
# change the column date dtype from object to date
data.Reported_Date= pd.to_datetime(data.Reported_Date, format='%Y/%m/%d %H:%M:%S%z')
data['Date']= data['Reported_Date'].dt.date
data['Time']= data['Reported_Date'].dt.time
data['Month']= data['Reported_Date'].dt.month
data['Year']= data['Reported_Date'].dt.year

# setting the index to be the date 
data.index = pd.DatetimeIndex(data.Reported_Date)
data=data.drop(columns=['Reported_Date'])
data

,NIBRS_Crime_Against,Offense_Category,Ward,Crime_Count,Date,Time,Month,Year
Reported_Date,,,,,,,,
2019-01-01 07:53:00+00:00,Person,Assault Offenses,10.0,1,2019-01-01,07:53:00,1,2019
2019-01-04 10:07:00+00:00,Property,Destruction/Damage/Vandalism of Property,3.0,1,2019-01-04,10:07:00,1,2019
2019-01-01 03:23:00+00:00,Property,Destruction/Damage/Vandalism of Property,1.0,1,2019-01-01,03:23:00,1,2019
2019-01-02 04:04:00+00:00,Property,Larceny/Theft Offenses,1.0,1,2019-01-02,04:04:00,1,2019
2019-01-04 05:27:00+00:00,Property,Stolen Property Offenses,4.0,1,2019-01-04,05:27:00,1,2019
...,...,...,...,...,...,...,...,...
2025-03-06 21:19:00+00:00,Property,Larceny/Theft Offenses,10.0,1,2025-03-06,21:19:00,3,2025
2025-03-07 13:04:00+00:00,Property,Larceny/Theft Offenses,12.0,1,2025-03-07,13:04:00,3,2025
2025-03-08 11:57:00+00:00,Property,Destruction/Damage/Vandalism of Property,8.0,1,2025-03-08,11:57:00,3,2025


### Examine the number of missing values 

In [672]:
# Examine the percent of missing data from each column
data.isnull().sum().sort_values(ascending=False)/len(data)*100


Ward                   0.785544
NIBRS_Crime_Against    0.000000
Offense_Category       0.000000
Crime_Count            0.000000
Date                   0.000000
Time                   0.000000
Month                  0.000000
Year                   0.000000
dtype: float64

In [673]:
# Remove missing Data
print(data.size)
data = data.dropna()
print(data.size)

2129480
2112752


### Step 6: Create a new CSV files for the Cleaned Dataset

In [674]:
#Reorder data
data = data.loc[:, ['Date','Year','Month', 'Time', 'Offense_Category','NIBRS_Crime_Against', 'Ward','Crime_Count']]

#create csv
data.to_csv(r"C:\Users\hajar\Time-Series-Crime-Forecasting-Minneapolis-3\data\Clean_Data.csv", index=False)
data

,Date,Year,Month,Time,Offense_Category,NIBRS_Crime_Against,Ward,Crime_Count
Reported_Date,,,,,,,,
2019-01-01 07:53:00+00:00,2019-01-01,2019,1,07:53:00,Assault Offenses,Person,10.0,1
2019-01-04 10:07:00+00:00,2019-01-04,2019,1,10:07:00,Destruction/Damage/Vandalism of Property,Property,3.0,1
2019-01-01 03:23:00+00:00,2019-01-01,2019,1,03:23:00,Destruction/Damage/Vandalism of Property,Property,1.0,1
2019-01-02 04:04:00+00:00,2019-01-02,2019,1,04:04:00,Larceny/Theft Offenses,Property,1.0,1
2019-01-04 05:27:00+00:00,2019-01-04,2019,1,05:27:00,Stolen Property Offenses,Property,4.0,1
...,...,...,...,...,...,...,...,...
2025-03-06 21:19:00+00:00,2025-03-06,2025,3,21:19:00,Larceny/Theft Offenses,Property,10.0,1
2025-03-07 13:04:00+00:00,2025-03-07,2025,3,13:04:00,Larceny/Theft Offenses,Property,12.0,1
2025-03-08 11:57:00+00:00,2025-03-08,2025,3,11:57:00,Destruction/Damage/Vandalism of Property,Property,8.0,1


### Step 8: Create a Univariate dataset with just Date and Occurences

In [675]:
#univariate dataset for date
Univ_Crime=data.drop(columns=['NIBRS_Crime_Against','Offense_Category','Time','Ward'])
Univ_Crime=Univ_Crime.groupby('Date')['Crime_Count'].sum()

#change series to dataframe
Univ_Crime=Univ_Crime.to_frame()

In [676]:
#create csv
Univ_Crime.to_csv(r"C:\Users\hajar\Time-Series-Crime-Forecasting-Minneapolis-3\data\Univ_Date.csv")
Univ_Crime

,Crime_Count
Date,
2019-01-01,88
2019-01-02,102
2019-01-03,133
2019-01-04,95
2019-01-05,83
...,...
2025-03-06,89
2025-03-07,89
2025-03-08,70
